In [3]:
import numpy as np
np.set_printoptions(suppress=True)
results_task1 = np.load('data/stateValuesTask1.npy')
grid_world = np.load('data/gridworld.npy')

In [4]:
grid_world

array([[ 10.,  10.,  10.,  -1.,  -1.,  -1.,  -1.,  10.,  10.],
       [ -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1., -20.,  10.],
       [ -1., -20., -20., -20., -20., -20.,  -1., -20.,  -1.],
       [ -1., -20.,  10.,  10.,  10.,  10., -20.,  -1., 100.],
       [ -1.,  -1.,  10., -20., -20., -20.,  -1.,  -1.,  -1.],
       [ -1.,  -1.,  -1.,  -1.,  -1.,  -1.,  -1., -20.,  -1.],
       [ -1., -20., -20., -20., -20., -20., -20.,  -1.,  -1.],
       [ -1.,  10.,  10.,  10.,  10.,  10.,  10.,  -1., -20.],
       [ -1.,  10.,  10.,  10., -20.,  10.,  10.,  -1., -20.]])

In [5]:
results_task1

array([[ -55.72285158,  -60.28813092,  -76.21943677,  -85.94239006,
         -95.17214143, -102.06460192, -109.14849891, -128.03672486,
        -154.35820755],
       [ -70.24180495,  -71.03860383,  -82.4266548 ,  -97.04877034,
        -113.45253199,    0.        , -138.83622028,    0.        ,
        -140.1279616 ],
       [-114.35635123,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.41838188,    0.        ,
         -71.58053207],
       [-113.49056343,    0.        ,  -67.69447722,  -82.24055051,
        -104.22849427, -131.61383646,    0.        ,   67.97543479,
           0.        ],
       [ -66.87372941,  -61.70095357,  -77.43127146,    0.        ,
           0.        ,    0.        ,  -51.56857317,  -41.72612892,
         -63.11723718],
       [ -64.4840581 ,  -63.86563014,  -69.53147116,  -77.8922909 ,
         -83.19180027,  -88.82103118,  -95.51685587,    0.        ,
        -145.95964503],
       [ -81.41499169,    0.        ,   

In [17]:
goal = {'x':8,'y':3}
THRESHOLD = 10

finished = False

def is_outside(state):
    return ((state['y'] < 0 or state['y'] >= grid_world.shape[0]) 
              or (state['x'] < 0 or state['x'] >= grid_world.shape[1]))

def terminal_state(state, goal_inc=True):
    # Entered a goal state
    if goal_inc and state == goal:
        return True
    # Exeeded boundary limits
    elif is_outside(state):
        return True
    # Entered a cell X
    elif grid_world[state['y'], state['x']] == -20:
        return True

    # Not a terminal state
    else:
        return False

def get_reward(state):
    if is_outside(state):
        return -30;
    else:
        return grid_world[state['y'], state['x']]

def make_move(state, action):
    state_c = state.copy()
    
    if action == 'l':
        state_c['x']  -= 1
    elif action == 'r':
        state_c['x']  += 1
    elif action == 'u':
        state_c['y']  -= 1
    elif action == 'd':
        state_c['y']  += 1
        
    if not terminal_state(state_c, goal_inc=False):
        state['x'] = state_c['x']
        state['y'] = state_c['y']
        
    return get_reward(state_c)

def compute_V(disc=0.9):
    # Value function matrix
    V = np.zeros_like(grid_world)
    
    # Delta value
    delta  = 0
    
    # Counter
    counter = 0
    
    # Threshold loop
    # while delta < THRESHOLD:
    for _ in range(65):
        # Reset delta
        delta  = 0
        
        # Copy of V
        V_copy = V.copy()
        
        # States loop
        for y in range(grid_world.shape[0]):
            for x in range(grid_world.shape[1]):
                # Current state
                state = {'x':x,'y':y}
                
                if terminal_state(state):
                    continue
                    
                counter += 1
                
                # Temporary value for the state
                V_prev = V[y,x]
                
                # Compute new value
                V[y,x] = 0
                for action, probability in [('r', 0.625), ('l', 0.125), ('u', 0.125), ('d', 0.125)]:
                    state_a = state.copy()
                    reward = make_move(state_a, action)
                    V[y,x] += probability * (reward + disc * V_copy[state_a['y'], state_a['x']])
                    #print("Action: ", state_a, reward, V_copy[state_a['y'], state_a['x']])
                
                # Compute new delta
                delta = max(delta, np.abs(V_prev - V[y,x]))
                #print("Delta: ", delta)
        
    return V

In [18]:
compute_V()

array([[ -55.67976943,  -60.24617709,  -76.17979713,  -85.90545978,
         -95.13827119, -102.03428114, -109.11976749, -128.01052852,
        -154.3333661 ],
       [ -70.19455692,  -70.99195666,  -82.38122703,  -97.00438911,
        -113.40877287,    0.        , -138.80056683,    0.        ,
        -140.10823911],
       [-114.30791483,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.37865377,    0.        ,
         -71.56941608],
       [-113.44516726,    0.        ,  -67.64485026,  -82.18743414,
        -104.1746153 , -131.5598189 ,    0.        ,   67.97705037,
           0.        ],
       [ -66.83495873,  -61.66282786,  -77.39141204,    0.        ,
           0.        ,    0.        ,  -51.55818152,  -41.71646341,
         -63.10637144],
       [ -64.451279  ,  -63.83843634,  -69.50892619,  -77.87552959,
         -83.17671707,  -88.80671264,  -95.50308696,    0.        ,
        -145.93781157],
       [ -81.36381986,    0.        ,   

In [12]:
results_task1

array([[ -55.72285158,  -60.28813092,  -76.21943677,  -85.94239006,
         -95.17214143, -102.06460192, -109.14849891, -128.03672486,
        -154.35820755],
       [ -70.24180495,  -71.03860383,  -82.4266548 ,  -97.04877034,
        -113.45253199,    0.        , -138.83622028,    0.        ,
        -140.1279616 ],
       [-114.35635123,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        , -156.41838188,    0.        ,
         -71.58053207],
       [-113.49056343,    0.        ,  -67.69447722,  -82.24055051,
        -104.22849427, -131.61383646,    0.        ,   67.97543479,
           0.        ],
       [ -66.87372941,  -61.70095357,  -77.43127146,    0.        ,
           0.        ,    0.        ,  -51.56857317,  -41.72612892,
         -63.11723718],
       [ -64.4840581 ,  -63.86563014,  -69.53147116,  -77.8922909 ,
         -83.19180027,  -88.82103118,  -95.51685587,    0.        ,
        -145.95964503],
       [ -81.41499169,    0.        ,   